In [ ]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 3 - 3.2 FaixaRiscoInternoAgrupada

### Geração de Massa

In [ ]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'solicitante.rendaLiquidaPicpay':[10000],
                             'payloadHomol.intermediarias.recorrenciaPrincipalidade':[0],
                             'payloadHomol.intermediarias.faixaScoreInternoCurto' :["R1"],
                             'solicitante.saldoMinimo30d':[0],
                             'payloadHomol.intermediarias.MATRecorrente':[0],
                             'solicitante.flagMAT':[0]
                            })

In [ ]:
#Criação das massas com casos forçados
massa_FaixaRiscoInternoAgrupada = sherlock.Gera_Massa(base_entrada)
massa_FaixaRiscoInternoAgrupada.gera_cenario('solicitante.rendaLiquidaPicpay',[-99999,600,3442.36,3442.37,3442.38,10000])
massa_FaixaRiscoInternoAgrupada.gera_cenario('payloadHomol.intermediarias.recorrenciaPrincipalidade',[0,1,2,3,4])
massa_FaixaRiscoInternoAgrupada.gera_cenario('payloadHomol.intermediarias.faixaScoreInternoCurto',["verificar","Sem info","R1","R2","R3","R4","R5","R6","R7","R8","R9","R10"])
massa_FaixaRiscoInternoAgrupada.gera_cenario('solicitante.saldoMinimo30d',[0,99,100,101,1000])
massa_FaixaRiscoInternoAgrupada.gera_cenario('payloadHomol.intermediarias.MATRecorrente',[0,1])
massa_FaixaRiscoInternoAgrupada.gera_cenario('solicitante.flagMAT',[0,1])

massa_FaixaRiscoInternoAgrupada.get_massa_final()

massa_FaixaRiscoInternoAgrupada.set_bom_arqv()
massa_FaixaRiscoInternoAgrupada.converte_massa()

### Execução Modular e Geração de CSV

In [ ]:
listaResultado = rp.execucaoModular(massa_FaixaRiscoInternoAgrupada.get_massa_json(),rp.FaixaRiscoInternoAgrupada)

##### Valida se todas condições estão sendo abordadas

In [ ]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['regraFaixaRiscoInternoAgrupada'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha = freqzinha.sort_values(by='valor', ascending=True)
pd.set_option("display.max_rows", 100)
freqzinha

In [ ]:
# Renomeação das colunas para o cabeçalho do RMA
massa_FaixaRiscoInternoAgrupada.rename_cabecalho()

In [ ]:
massa_FaixaRiscoInternoAgrupada.get_massa_dm()

In [ ]:
massa_FaixaRiscoInternoAgrupada.atribui_expected(listaResultado,"payloadHomol.intermediarias.faixaRiscoInternoAgrupada","rangeScoreInterno")
massa_FaixaRiscoInternoAgrupada.get_massa_dm()

### Conversão da massa para CSV

In [ ]:
massa_FaixaRiscoInternoAgrupada.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaFaixaRiscoInternoAgrupada.csv',index=False)

### Análise divergências

In [ ]:
dataid = 44
print(json.dumps(listaResultado[dataid-1],indent=4))